In [22]:
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.preprocessing_schoolid import SchoolYear
from src.cps_model import print_cv_results

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The preprocessing is handled by the SchoolYear class and its methods.

In [23]:
sy_1819 = SchoolYear('../../data/chicago_data_portal_csv_files/Chicago_Public_Schools_-_School_Profile_Information_SY1819.csv',
                     '../../data/chicago_data_portal_csv_files/.ipynb_checkpoints/Chicago_Public_Schools_-_School_Progress_Reports_SY1819-checkpoint.csv')

The original data set has 651 schools. 

In [24]:
sy_1819.merged_df.shape

(651, 276)

In [25]:
sy_1819.isolate_high_schools()
sy_1819.drop_no_gr_schools()
sy_1819.drop_no_student_schools()
sy_1819.make_percent_low_income()


,School_ID,Legacy_Unit_ID,Finance_ID,Short_Name_sp,Long_Name_sp,Primary_Category_sp,Is_High_School,Is_Middle_School,Is_Elementary_School,Is_Pre_School,Summary,Administrator_Title,Administrator,Secondary_Contact_Title,Secondary_Contact,Address_sp,City_sp,State_sp,Zip_sp,Phone_sp,Fax_sp,CPS_School_Profile_sp,Website_sp,Facebook,Twitter,Youtube,Pinterest,Attendance_Boundaries,Grades_Offered_All,Grades_Offered,Student_Count_Total,Student_Count_Low_Income,Student_Count_Special_Ed,Student_Count_English_Learners,Student_Count_Black,Student_Count_Hispanic,Student_Count_White,Student_Count_Asian,Student_Count_Native_American,Student_Count_Other_Ethnicity,Student_Count_Asian_Pacific_Islander,Student_Count_Multi,Student_Count_Hawaiian_Pacific_Islander,Student_Count_Ethnicity_Not_Available,Statistics_Description,Demographic_Description,Dress_Code,PreK_School_Day,Kindergarten_School_Day,School_Hours,Freshman_Start_End_Time,After_School_Hours,Earliest_Drop_Off_Time,Classroom_Languages,Bilingual_Services,Refugee_Services,Title_1_Eligible,PreSchool_Inclusive,Preschool_Instructional,Significantly_Modified,Hard_Of_Hearing,Visual_Impairments,Transportation_Bus,Transportation_El,Transportation_Metra,School_Latitude_sp,School_Longitude_sp,Average_ACT_School,Mean_ACT,College_Enrollment_Rate_School,College_Enrollment_Rate_Mean,Graduation_Rate_School,Graduation_Rate_Mean,Overall_Rating,Rating_Status,Rating_Statement,Classification_Description,School_Year,Third_Contact_Title,Third_Contact_Name,Fourth_Contact_Title,Fourth_Contact_Name,Fifth_Contact_Title,Fifth_Contact_Name,Sixth_Contact_Title,Sixth_Contact_Name,Seventh_Contact_Title,Seventh_Contact_Name,Network,Is_GoCPS_Participant,Is_GoCPS_PreK,Is_GoCPS_Elementary,Is_GoCPS_High_School,Open_For_Enrollment_Date,Closed_For_Enrollment_Date,Short_Name_pr,Long_Name_pr,School_Type,Primary_Category_pr,Address_pr,City_pr,State_pr,Zip_pr,Phone_pr,Fax_pr,CPS_School_Profile_pr,Website_pr,Progress_Report_Year,Blue_Ribbon_Award_Year,Excelerate_Award_Gold_Year,Spot_Light_Award_Year,Improvement_Award_Year,Excellence_Award_Year,Student_Growth_Rating,Student_Growth_Description,Growth_Reading_Grades_Tested_Pct_ES,Growth_Reading_Grades_Tested_Label_ES,Growth_Math_Grades_Tested_Pct_ES,Growth_Math_Grades_Tested_Label_ES,Student_Attainment_Rating,Student_Attainment_Description,Attainment_Reading_Pct_ES,Attainment_Reading_Lbl_ES,Attainment_Math_Pct_ES,Attainment_Math_Lbl_ES,Culture_Climate_Rating,Culture_Climate_Description,School_Survey_Student_Response_Rate_Pct,School_Survey_Student_Response_Rate_Avg_Pct,School_Survey_Teacher_Response_Rate_Pct,School_Survey_Teacher_Response_Rate_Avg_Pct,School_Survey_Parent_Response_Rate_Pct,School_Survey_Parent_Response_Rate_Avg_Pct,Healthy_School_Certification,Healthy_School_Certification_Description,Creative_School_Certification,Creative_School_Certification_Description,NWEA_Reading_Growth_Grade_3_Pct,NWEA_Reading_Growth_Grade_3_Lbl,NWEA_Reading_Growth_Grade_4_Pct,NWEA_Reading_Growth_Grade_4_Lbl,NWEA_Reading_Growth_Grade_5_Pct,NWEA_Reading_Growth_Grade_5_Lbl,NWEA_Reading_Growth_Grade_6_Pct,NWEA_Reading_Growth_Grade_6_Lbl,NWEA_Reading_Growth_Grade_7_Pct,NWEA_Reading_Growth_Grade_7_Lbl,NWEA_Reading_Growth_Grade_8_Pct,NWEA_Reading_Growth_Grade_8_Lbl,NWEA_Math_Growth_Grade_3_Pct,NWEA_Math_Growth_Grade_3_Lbl,NWEA_Math_Growth_Grade_4_Pct,NWEA_Math_Growth_Grade_4_Lbl,NWEA_Math_Growth_Grade_5_Pct,NWEA_Math_Growth_Grade_5_Lbl,NWEA_Math_Growth_Grade_6_Pct,NWEA_Math_Growth_Grade_6_Lbl,NWEA_Math_Growth_Grade_7_Pct,NWEA_Math_Growth_Grade_7_Lbl,NWEA_Math_Growth_Grade_8_Pct,NWEA_Math_Growth_Grade_8_Lbl,NWEA_Reading_Attainment_Grade_2_Pct,NWEA_Reading_Attainment_Grade_2_Lbl,NWEA_Reading_Attainment_Grade_3_Pct,NWEA_Reading_Attainment_Grade_3_Lbl,NWEA_Reading_Attainment_Grade_4_Pct,NWEA_Reading_Attainment_Grade_4_Lbl,NWEA_Reading_Attainment_Grade_5_Pct,NWEA_Reading_Attainment_Grade_5_Lbl,NWEA_Reading_Attainment_Grade_6_Pct,NWEA_Reading_Attainment_Grade_6_Lbl,NWEA_Reading_Attainment_Grade_7_Pct,NWEA_Readi

After dropping K-8 schools, schools with no graduation rates or student counts, and schools with non-traditional missions, the dataset is left with 124 schools. 

In [26]:
sy_1819.merged_df.shape

(134, 277)

# Train Test Split


For test size, I will isolate 20% of the data for test data. During cross validation, I will create 4 folds out of 100, so that each fold will have just about the same number of records as the holdout set.

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sy_1819.merged_df.drop("Graduation_Rate_School", axis=1),
                                                   sy_1819.merged_df["Graduation_Rate_School"], test_size=.2, 
                                                   random_state=42)

# FSM: Mean Prediction

## Mean

In [41]:
from sklearn.metrics import mean_squared_error, r2_score

y_hat_mean = np.full(y_train.shape, np.mean(y_train))

print('R2: Simple prediction of mean')
print(r2_score(y_train,y_hat_mean ))


print('RMSE: Simple prediction of mean')
print(mean_squared_error(y_train,y_hat_mean, squared=False))

R2: Simple prediction of mean
0.0
RMSE: Simple prediction of mean
21.494672805970403


# Linear Regression

## Simple Linear Regression

The FSM only includes one predictor: `Student_Count_Total`, which has a high relative correlation to the target.  

In [44]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

In [45]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

y_hat = print_cv_results(lr, X_train, ['Student_Count_Total'], y_train )

#####r2#####
0.20248042617957038
test
[0.1012748  0.1635617  0.09850899]
0.12111516273655187
#####rmse#####
#####Train####
[18.87383651 19.05671021 19.44464238]
19.125063030990777
test
[20.1795537  20.439211   19.09257922]
19.90378130857657
cross val predict metrics: test scores
RMSE
19.91982921575195
r2
0.1411653774043521


## Add perc_low_income_students

In [46]:
lr = LinearRegression()

y_hat = print_cv_results(lr, X_train, ['Student_Count_Total', 'perc_low_income'], y_train )

#####r2#####
0.21790299199412003
test
[0.12840802 0.18193494 0.06837706]
0.126240008169364
#####rmse#####
#####Train####
[18.77693702 18.96632094 19.06150962]
18.934922530708945
test
[19.87260084 20.21348022 19.40903698]
19.83170601003629
cross val predict metrics: test scores
RMSE
19.838378915667313
r2
0.14817440571746188


# Decision Tree Regressor

In [31]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()

y_hat_dt = print_cv_results(dt,X_train, ['Student_Count_Total'], y_train )

#####r2#####
0.9569634244212804
test
[-0.87804874  0.04920533 -0.22270645]
-0.3505166206458689
#####rmse#####
#####Train####
[4.33678176 6.16672694 0.        ]
3.501169566833943
test
[29.17101396 21.79167145 22.23540292]
24.3993627749896
cross val predict metrics: test scores
RMSE
24.653516744076644
r2
-0.3155158352967107


## Add perc_low_income

In [32]:
y_hat = print_cv_results(dt, X_train, ['Student_Count_Total', 'perc_low_income'], y_train )

#####r2#####
1.0
test
[-0.37391673  0.12940757 -0.15425678]
-0.13292198057808804
#####rmse#####
#####Train####
[0. 0. 0.]
-0.0
test
[24.95044533 20.85233134 21.60404724]
22.468941302945865
cross val predict metrics: test scores
RMSE
23.86268936908063
r2
-0.2324721049747991


# Random Forest Regressor

In [33]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()

y_hat_dt = print_cv_results(rf, X_train, ['Student_Count_Total'], y_train )

#####r2#####
0.8440094884061135
test
[-0.39325236  0.22626595  0.0465101 ]
-0.04015877030402385
#####rmse#####
#####Train####
[7.22076718 9.72006084 8.11024573]
8.350357915025105
test
[25.12540026 19.65816737 19.63549934]
21.473022326345145
cross val predict metrics: test scores
RMSE
21.29956317509663
r2
0.01807183726733408


## Add perc_low_income

In [20]:
y_hat = print_cv_results(rf, X_train, ['Student_Count_Total', 'perc_low_income'], y_train )

#####r2#####
0.887703200935308
test
[0.22109443 0.33222837 0.19254691]
0.24862323359775132
#####rmse#####
#####Train####
[5.8690869  8.00760597 7.41502627]
7.097239711958789
test
[18.78626746 18.26253898 18.06935094]
18.372719129066414
cross val predict metrics: test scores
RMSE
18.407989991650645
r2
0.266582848882165


# Best so far: 18.3727 RMSE

# Inverse Log transform target

In [15]:
y_hat_train = print_cv_results(lr,X_train, ['Student_Count_Total'], np.log(100-y_train) )

#####r2#####
0.24332259579712523
test
[0.1310171  0.21155901 0.11198908]
0.1515217300353819
#####rmse#####
#####Train####
[0.67627829 0.72975271 0.6978114 ]
0.7012808018773812
test
[0.77365276 0.70009198 0.73488376]
0.7362095011457256
cross val predict metrics: test scores
RMSE
0.7368404420480538
r2
0.17029449357156934


In [17]:
from sklearn.metrics import mean_squared_error, r2_score
y_hat_train_untransformed = 100 - np.e**y_hat_train

rmse = mean_squared_error(y_train, y_hat_train_untransformed, squared=False)
print(rmse)

r2 = r2_score(y_train, y_hat_train_untransformed)
print(r2)

20.156315026940735
0.12065236972536697


In [ ]:
from src.cps_model import cv_cps

%load_ext autoreload
%autoreload 2

In [ ]:
cv_cps(lr, X_train, y_train, features=['Student_Count_Total'])

The first simple model is high bias.  Only ~21% of the variance is explained.  The test score R^2 have a comprable mean R2, but have a high standard deviation.



In this simple model, for every 8 students, for every 100 students, the graduation rate goes up .84 points.  

## Add Low Income Students

In [ ]:
cv_cps(lr, X_train, y_train, ['Student_Count_Total', 'Student_Count_Low_Income'])

Adding low income student count makes the test scores drop significantly.

## Add Percent Low Income Students

In [ ]:
cv_cps(lr, X_train, y_train, ['Student_Count_Total', 'perc_low_income'])

Percent low income has a positive effect on both the training and test scores.  

One test set is performing significantly worse than the ohers at .14 R^2.

## One Hot Encode Networks

In [ ]:
from sklearn.impute import SimpleImputer

network_imputer = SimpleImputer(strategy='constant', fill_value='missing')

X_train['Network'] = network_imputer.fit_transform(X_train[['Network']])

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder


# network_imputer = ColumnTransformer([('network_impute', SimpleImputer(strategy='constant', fill_value='missing'), ['Network'])], 
#                                          remainder='passthrough')


# network_transformer = ColumnTransformer( [('network_impute', SimpleImputer(strategy='constant', fill_value='missing'), ['Network']), 
#                                          ('network_ohe', OneHotEncoder(handle_unknown='ignore'), ['Network'])], 
#                                          remainder='passthrough')


network_ohe = ColumnTransformer([ 
                                          ('network_ohe', OneHotEncoder(handle_unknown='ignore'), ['Network'])], 
                                         remainder='passthrough')

network_pipe = make_pipeline(network_ohe, LinearRegression())

cv_cps(network_pipe, X_train, y_train, ['Student_Count_Total', 'perc_low_income', 'Network'])

Adding Network One Hot Encoded columns decreases the bias, but increases the vaiance.  There is even one negative R^2 score which is particularly worrisome.

# High Correlation 

In [ ]:
df_train.corr().iloc[-1].dropna().sort_values()

In [ ]:
df_train.corr().iloc[-1].dropna().sort_values(ascending=False)

In [ ]:
df_train[['Graduation_Rate_School', 'Student_Count_Low_Income']].corr()